### All code for CMC placed here, detailed with markdown. 

In [31]:
import requests
import pprint
import requests
from requests.exceptions import HTTPError
from datetime import datetime as dt
import pandas as pd
import numpy as np
from path import Path

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pprint
import os
from coinmarketcap import Market
from prettytable import PrettyTable

### WrapAPI

In [2]:
for url in ["https://wrapapi.com/use/eccadena/trackico/finances1/0.0.1?wrapAPIKey=HzqWb8NGMMD0VcX0CF4VWYvcle67YWC5"]:
    try:
        response = requests.get(url)

        # If the response was successful, no Exception will be raised
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  # Python 3.6
    except Exception as err:
        print(f'Other error occurred: {err}')  # Python 3.6
    else:
        print('Success!')

Success!


In [3]:
response = requests.get(f"https://wrapapi.com/use/eccadena/trackico/finances1/0.0.1?wrapAPIKey=HzqWb8NGMMD0VcX0CF4VWYvcle67YWC5")
response = response.json()
type(response)

dict

In [4]:
response

{'success': True,
 'outputScenario': 'Default',
 'data': {'output': ['04/18/2019 midnight\n- Unspecified',
   'Switzerland',
   'ICO',
   'ICO',
   'INBIT',
   'Ethereum',
   '0.01 EUR\n\n(0.01103 USD)',
   '6,500,000,000 INBIT\n (51%)',
   '12,763,636,364 INBIT',
   '50,000,000\nEUR\n (55,150,000 USD)',
   'ETH, BTC, LTC, BCH',
   'Afghanistan, Bosnia and Herzegovina, Central African Republic, Congo, Cuba, Eritrea, Ethiopia, Guinea-Bissau, Iran, Iraq, Lebanon, Libya, North Korea, Somalia, South Sudan, Sudan, Switzerland, Syria, Uganda, United States of America, Vanuatu, Yemen',
   'Yes',
   'Presale Tier 5Period: 07.10.2019 - 27.10.2019Discount: 20%Amount of sale: 250 000 €',
   'Dec. 19, 2018',
   'May 5, 2019',
   '133,333,000 InBit',
   '1,470,662 USD',
   '1.0 %',
   '04/18/2019 midnight\n- Unspecified',
   'Switzerland',
   'ICO',
   'Ethereum',
   '0.01 EUR',
   'Available',
   '694,420 th',
   '8,929',
   '4,933']},
 'stateToken': 'eyJqYXIiOnsidmVyc2lvbiI6InRvdWdoLWNvb2tpZUAyLj

In [5]:
# different url/wrap... attempt at bring in all cryptos from CMCAPI
response = requests.get(f"https://wrapapi.com/use/eccadena/cmcapi/all/0.0.1?wrapAPIKey=HzqWb8NGMMD0VcX0CF4VWYvcle67YWC5")
response = response.json()
type(response)

dict

In [9]:
# function to create a df from CMC api all crypto call
def CMC_api_all():
    
    df_all = pd.DataFrame()
    
    l = list(range(20000))
    
    for i in l[0::11]:
        df_raw = pd.DataFrame(response['data']['output'][0+i:11+i]).T
        df_all = pd.concat([df_raw,df_all])
        # print('Joining Dataframe')
    return df_all    

df_all_CMC = CMC_api_all()
df_all_CMC.rename(columns = {'0':'rank_cmc','1':'title','2':'ticker','3':'market_cap','4':'price','5':'supply','6':'volume24','7':'pct1h','8':'pct24h','9':'pct7d','10':'addremove'}, 
                  inplace = True) #unsure why this will not work. tried inside the function

In [11]:
df_all_CMC.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1819,PKB\n\nParkByte,PKB,"$17,605",$0.003695,"4,764,026\n*",Low Vol,0.00%,0.00%,-4.74%,Add to Watchlist\nRemove from Watchlist\nWatch...
0,1818,PAK\n\nPakcoin,PAK,"$17,746",$0.000259,"68,594,750",Low Vol,-0.60%,-26.45%,-6.63%,Add to Watchlist\nRemove from Watchlist\nWatch...
0,1817,IQ\n\nIQ.cash,IQ,"$17,808",$0.004571,"3,896,163",Low Vol,-0.60%,48.53%,12.57%,Add to Watchlist\nRemove from Watchlist\nWatch...
0,1816,BND\n\nBlocknode,BND,"$18,118",$0.000086,"210,095,225",Low Vol,-0.60%,-1.92%,-10.47%,Add to Watchlist\nRemove from Watchlist\nWatch...
0,1815,DMB\n\nDigital Money...,DMB,"$18,206",$0.000172,"105,560,695",Low Vol,-0.60%,-34.61%,-46.24%,Add to Watchlist\nRemove from Watchlist\nWatch...


## To run the cell below go to https://coinmarketcap.com/all/views/all/
## Copy all cryptos onto clipboard
## This is done as the Free API calls seem to be limited and random. Free api is depricated.

In [ ]:
# DO NOT RUN AGAIN UNLESS YOU HAVE COPIED THE WEBSITE df_all_CMC_clipboard = pd.read_clipboard()

In [ ]:
df_all_CMC_clipboard.drop(columns = ['Unnamed: 10'], inplace = True)

In [ ]:
df_all_CMC_clipboard.shape

In [ ]:
df_all_CMC_clipboard.head()

In [ ]:
def CMC_df_cleaning(df):
    
    df['price'] = df['price'].apply(lambda x: x.strip('$'))
    df['market_cap'] = df['market_cap'].apply(lambda x: x.strip('$'))
    df['volume24'] = df['volume24'].apply(lambda x: x.strip('$'))
    
    df['pct1h'] = df['pct1h'].apply(lambda x: x.strip('%'))
    df['pct24h'] = df['pct24h'].apply(lambda x: x.strip('%'))
    df['pct7d'] = df['pct7d'].apply(lambda x: x.strip('%'))
    
    df['supply'] = df['pct7d'].apply(lambda x: x.strip('*'))
    
    df = df.replace('?',0)
    
    return df

In [ ]:
df_all_CMC = CMC_df_cleaning(df_all_CMC_clipboard)

In [ ]:
df_all_CMC.head()

In [ ]:
file_path = Path("../../../../data/cleandata/CMC_df.csv")
df_all_CMC.to_csv(file_path, index=False)

### Example code from SandBox. This code is in Python 2... abandoned after it was realized the FREE api will only allow for 333 calls. Have code here for future use if we decide to use this API in a function... must conver to python 3

In [14]:
API_key = os.getenv('CMC_API')

In [15]:
url = 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'

API_key = os.getenv('CMC_API')

parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': API_key,
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

{'status': {'timestamp': '2019-11-15T03:08:29.344Z', 'error_code': 401, 'error_message': 'This API Key is invalid.', 'elapsed': 0, 'credit_count': 0}}


In [16]:
url = 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/map'
parameters = {
  'symbol':'BTC,USDT,BNB'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': API_key,
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  pprint.pprint(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

{'status': {'credit_count': 0,
            'elapsed': 0,
            'error_code': 401,
            'error_message': 'This API Key is invalid.',
            'timestamp': '2019-11-15T03:08:34.511Z'}}


In [18]:
#Historical Data
url = 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest'
parameters = {
  'id':'1',
  'time_start':'2018-01-01',
  'time_end':'2019-01-03'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': API_key,
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  pprint.pprint(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

{'status': {'credit_count': 0,
            'elapsed': 0,
            'error_code': 401,
            'error_message': 'This API Key is invalid.',
            'timestamp': '2019-11-15T03:08:58.118Z'}}


# Calling crupto currencies with depricated API. As of 11/14/2019 this code will still call a large amount of Cryptos

## URL - https://pypi.org/project/coinmarketcap/

In [19]:
coinmarketcap = Market()

In [20]:
cryptos_available = pd.DataFrame.from_dict(coinmarketcap.listings()['data'])
cryptos_available.head()

,id,name,symbol,website_slug
0,1,Bitcoin,BTC,bitcoin
1,2,Litecoin,LTC,litecoin
2,3,Namecoin,NMC,namecoin
3,4,Terracoin,TRC,terracoin
4,5,Peercoin,PPC,peercoin


In [21]:
pd.DataFrame.from_dict(coinmarketcap.ticker(start=0, limit=3, convert='USD')['data']).T

,circulating_supply,id,last_updated,max_supply,name,quotes,rank,symbol,total_supply,website_slug
1,1.8048e+07,1,1573787375,2.1e+07,Bitcoin,"{'USD': {'price': 8617.9398866, 'volume_24h': ...",1,BTC,1.8048e+07,bitcoin
1027,1.08557e+08,1027,1573787422,None,Ethereum,"{'USD': {'price': 183.525444117, 'volume_24h':...",2,ETH,1.08557e+08,ethereum
52,4.32985e+10,52,1573787403,1e+11,XRP,"{'USD': {'price': 0.2670287203, 'volume_24h': ...",3,XRP,9.99913e+10,ripple


In [23]:
# pd.DataFrame.from_dict(coinmarketcap.ticker(start=0, limit=3, convert='USD')['data']['1']['quotes']['USD'])

In [24]:
df_cmc_USD = pd.DataFrame(coinmarketcap.ticker(start=0, limit=3, convert='USD')['data']['1']['quotes']['USD'].items())
df_cmc_USD.set_index(0).T

,price,volume_24h,market_cap,percent_change_1h,percent_change_24h,percent_change_7d
1,8617.939887,1.973485e+10,1.555368e+11,-0.35,-1.95,-6.68


In [25]:
df_cmc_USD = pd.DataFrame(coinmarketcap.ticker(start=0, limit=3, convert='USD')['data']['1']['quotes']['USD'].items()).T
# df_cmc_USD = df_cmc_USD.reindex(['price', 'volume_24h', 'market_cap', 'percent_change_1h', 'percent_change_24h', 'percent_change_7d'], axis=1)

df_cmc_USD

,0,1,2,3,4,5
0,price,volume_24h,market_cap,percent_change_1h,percent_change_24h,percent_change_7d
1,8617.94,1.97349e+10,1.55537e+11,-0.35,-1.95,-6.68


### Call below is cached at 120 seconds. Not an issue for a sleep timer.
### However each call is the same tickers... documentation does not show how to call new tickers

In [ ]:
api = "https://api.coinmarketcap.com/v2/ticker/"
raw_data = requests.get(api).json()
data = raw_data['data']

df = pd.DataFrame()

for currency in data:
    name = data[currency]['name']
    price = data[currency]['quotes']['USD']['price']
    change_1h = data[currency]['quotes']['USD']['percent_change_1h']
    change_24h = data[currency]['quotes']['USD']['percent_change_24h']
    change_7d = data[currency]['quotes']['USD']['percent_change_7d']
    
    df_2 = pd.DataFrame({'name': [name], 'price': [price], 'change_1h': [change_1h], 'change_24h': [change_24h], 'change_7d': [change_7d]})
    
    df = df.append(df_2, ignore_index=True)
    
df

In [28]:
df.head()

,name,price,change_1h,change_24h,change_7d
0,Bitcoin,8617.688003,-0.35,-1.95,-6.68
1,Ethereum,183.493907,-0.99,-2.26,-2.01
2,XRP,0.266968,-0.28,-2.19,-7.38
3,Bitcoin Cash,273.165148,-1.11,-4.26,-6.54
4,Tether,1.007406,-0.15,0.34,0.42


In [33]:
# experiment with pretty table
api = "https://api.coinmarketcap.com/v2/ticker/"
raw_data = requests.get(api).json()
data = raw_data['data']

table = PrettyTable()

for currency in data:
    name = data[currency]['name']
    symbol = data[currency]['symbol']
    rank = data[currency]['rank']
    supply = data[currency]['circulating_supply']
    price = data[currency]['quotes']['USD']['price']
    change_1h = data[currency]['quotes']['USD']['percent_change_1h']
    change_24h = data[currency]['quotes']['USD']['percent_change_24h']
    change_7d = data[currency]['quotes']['USD']['percent_change_7d']
    

    table.add_row([name, price, change_1h, change_24h, change_7d, symbol, supply, rank])

table.field_names = ["Name","Price","Change 1h","Change 24h","Change 7d",'symbol', 'supply', 'rank']
table.sortby = "Change 24h"
table.reversesort = True
print(table)

+-------------------------+---------------+-----------+------------+-----------+--------+----------------+------+
|           Name          |     Price     | Change 1h | Change 24h | Change 7d | symbol |     supply     | rank |
+-------------------------+---------------+-----------+------------+-----------+--------+----------------+------+
|      Maximine Coin      |  0.0589737828 |   -1.26   |   41.03    |   182.0   |  MXM   |  1649000000.0  |  63  |
|          Seele          |  0.1355270633 |   -1.15   |   21.77    |   65.35   | SEELE  |  696705193.0   |  66  |
|         VeChain         |  0.0071981414 |    -1.4   |    14.0    |   26.14   |  VET   | 55454734800.0  |  27  |
|          Beldex         |  0.0747463798 |    0.98   |   13.66    |   10.15   |  BDX   |  980222595.0   |  85  |
|          Verge          |  0.0044191371 |    0.22   |   12.44    |   11.42   |  XVG   | 16051236999.0  |  89  |
|          Tezos          |  1.2463500764 |    0.61   |   10.76    |    2.96   |  XTZ   

In [36]:
# experiment with pretty table
api = "https://api.coinmarketcap.com/v2/ticker/"
raw_data = requests.get(api).json()
data = raw_data['data']

cmc_api_depricated = pd.DataFrame()

for currency in data:
    name = data[currency]['name']
    symbol = data[currency]['symbol']
    rank = data[currency]['rank']
    supply = data[currency]['circulating_supply']
    price = data[currency]['quotes']['USD']['price']
    change_1h = data[currency]['quotes']['USD']['percent_change_1h']
    change_24h = data[currency]['quotes']['USD']['percent_change_24h']
    change_7d = data[currency]['quotes']['USD']['percent_change_7d']
    
    df = pd.DataFrame([{'name': name,'symbol': symbol,'rank': rank,'supply': supply,'price': price,'pct1h': change_1h,'pct24h':change_24h ,'pct7d':change_7d}])
    
    cmc_api_depricated = cmc_api_depricated.append(df, ignore_index=True)


In [37]:
cmc_api_depricated

,name,pct1h,pct24h,pct7d,price,rank,supply,symbol
0,Bitcoin,-0.03,-1.69,-6.46,8636.546291,1,1.804804e+07,BTC
1,Ethereum,-0.66,-1.97,-1.76,183.941598,2,1.085567e+08,ETH
2,XRP,-0.12,-2.04,-7.22,0.267280,3,4.329848e+10,XRP
3,Bitcoin Cash,-0.53,-3.66,-6.01,274.634624,4,1.811354e+07,BCH
4,Tether,-0.05,0.38,0.46,1.007853,5,4.108044e+09,USDT
5,Litecoin,-0.40,-2.68,-4.28,59.306793,6,6.368168e+07,LTC
6,Binance Coin,-0.27,-2.07,1.78,20.895518,7,1.555367e+08,BNB
7,EOS,-0.24,-1.75,-2.93,3.405506,8,9.402111e+08,EOS
8,Bitcoin SV,0.04,-2.18,-5.25,125.469624,9,1.806842e+07,BSV
9,Stellar,-0.28,-3.10,-2.17,0.073693,10,2.005478e+10,XLM
